In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Replace '/content/drive/My Drive/MyProject/requirements.txt'
# with the actual path to YOUR requirements.txt file
requirements_path = '/content/drive/My Drive/requirements.txt'

# Run pip install using the full path
!pip install -q -r "{requirements_path}"

In [15]:
# global QUALIFIRE_API_KEY
# QUALIFIRE_API_KEY = "" #@param {type:"string"}

# global OPENAI_API_KEY
# OPENAI_API_KEY = "" #@param {type:"string"}

In [16]:
import qualifire
import os

if QUALIFIRE_API_KEY == "YOUR QUALIFIRE API KEY":
    print("Please replace 'YOUR QUALIFIRE API KEY' with your actual key.")
else:
    qualifire.init(
        api_key=QUALIFIRE_API_KEY,
    )
    print("Qualifire SDK Initialized Successfully!")

Qualifire SDK Initialized Successfully!


In [17]:
if OPENAI_API_KEY == "YOUR OPENAI API KEY":
    print("Please replace 'YOUR OPENAI API KEY' with your actual key for the agent to run.")
else:
    print("OpenAI API Key set.")
# Note: The agent code itself will pass this to the init_chat_model function.
# Setting it as an environment variable here is good practice if other parts of LangGraph expect it.
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

OpenAI API Key set.


In [19]:
# Install necessary libraries if they are not already installed
!pip install -q langchain langchain-community langchain-core langgraph openai

In [33]:
import requests
import os # Make sure os is imported for path handling

# New import for OpenAI models in LangChain
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
# We will build the graph manually, so we don't need create_react_agent directly
# from langgraph.prebuilt import create_react_agent

print("Imported necessary libraries for the agent setup.")

# API Key check - this block is now redundant if you used Cell 2 to load keys
# but leaving a modified version for safety, assuming keys are now in local scope.
if 'QUALIFIRE_API_KEY' not in locals() or 'OPENAI_API_KEY' not in locals() or \
   not QUALIFIRE_API_KEY or not OPENAI_API_KEY:
    print("\nERROR: API keys are not set. Please ensure QUALIFIRE_API_KEY and OPENAI_API_KEY are loaded.\n")
else:
    print("\nAPI keys seem to be set. Proceeding with agent initialization.\n")

    # Initialize the LLM using ChatOpenAI from langchain_openai
    llm = ChatOpenAI(
        model="gpt-4", # 'gpt-4.1' is not a standard OpenAI model name, use 'gpt-4' or 'gpt-4-turbo' etc.
                       # Qualifire can proxy standard OpenAI models.
        api_key=OPENAI_API_KEY,
        base_url="https://proxy.qualifire.ai/api/providers/openai/",
        default_headers={
            "X-Qualifire-API-Key": QUALIFIRE_API_KEY
        },
        # You might also want to add temperature=0 for more consistent responses
        temperature=0
    )
    print("LLM Initialized.")

    # Download the Chinook database
    # Since you've used %cd in Cell 3, "./assets/Chinook.db" should now correctly
    # point to /content/drive/My Drive/MyProject/assets/Chinook.db
    db_file_path = "/content/drive/My Drive/Chinook.db"

    # Add a check to ensure the file exists before trying to load it
    if not os.path.exists(db_file_path):
        print(f"CRITICAL ERROR: Database file not found at {db_file_path}")
        print("Please ensure 'Chinook.db' is inside an 'assets' folder within your project folder in Google Drive.")
        # Exit or raise an error as the script cannot proceed without the DB
        raise FileNotFoundError(f"Database file not found: {db_file_path}")

    # Setup SQL Database
    db = SQLDatabase.from_uri(f"sqlite:///{db_file_path}")
    print(f"SQL Database loaded from: {db_file_path}")


    # Setup SQL Database Toolkit
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    tools = toolkit.get_tools()
    print("\nAvailable Tools:")
    for tool in tools:
        print(f"- {tool.name}: {tool.description}")

    # Define the system prompt for the agent
    system_prompt_string = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
        dialect=db.dialect,
        top_k=5,
    )

    # --- Start Manual LangGraph Agent Construction ---

    from langgraph.graph import StateGraph, END
    from langgraph.prebuilt import ToolNode
    from typing import TypedDict, Annotated, Sequence
    from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

    # Define the state
    class AgentState(TypedDict):
        messages: Annotated[Sequence[BaseMessage], lambda x, y: x + y]

    # Define the agent node
    # This node will call the LLM and decide the next action
    def run_agent(state):
        messages = state['messages']
        # Bind the tools to the LLM for function calling
        agent = llm.bind_tools(tools)
        response = agent.invoke(messages)
        return {"messages": [response]}

    # Define the tool node
    # This node will execute the selected tool
    tool_node = ToolNode(tools)

    # Define the decision logic
    # This function will determine if the agent should use a tool or end
    def should_continue(state):
        messages = state['messages']
        last_message = messages[-1]
        # If the last message is a tool call, then we run the tool node
        if last_message.tool_calls:
            return "continue" # or the name of the tool node, e.g., "tools"
        # Otherwise, we end the graph
        else:
            return "end"

    # Build the graph
    workflow = StateGraph(AgentState)

    # Add nodes
    workflow.add_node("agent", run_agent)
    workflow.add_node("tools", tool_node) # Name the tool node "tools"

    # Set the entry point
    workflow.set_entry_point("agent")

    # Add edges
    # From agent to decision (should_continue)
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "continue": "tools", # If continue, go to the tools node
            "end": END # If end, finish the graph
        }
    )

    # From tools back to the agent node to process the tool output
    workflow.add_edge('tools', 'agent')


    # Compile the graph
    agent_executor = workflow.compile()

    print("\nAgent executor created (manual graph construction).")

    # --- End Manual LangGraph Agent Construction ---


    # Define a question and stream the agent's response
    question = "Which sales agent made the most in sales in 2009?"
    print(f"\nQuestion: {question}\n")
    print("Streaming agent response:")
    print("-" * 30)

    try:
        # The input to the compiled graph needs to match the state structure
        for step in agent_executor.stream(
            {"messages": [HumanMessage(content=question)]}, # Input is a list of messages
            stream_mode="values", # "values" provides the full state at each step
        ):
            # The 'messages' key contains a list of AIMessage, HumanMessage, etc.
            # We're interested in the last message, which is usually the agent's thought or response.
            if "messages" in step and step["messages"]:
                step["messages"][-1].pretty_print()
                print("-" * 30)
    except Exception as e:
        print(f"An error occurred during agent execution: {e}")
        print("This might be due to API key issues, network problems, or unexpected agent behavior.")
        print("Please check your API keys and network connection.")
        print("If the agent itself has an issue, the Qualifire traces (if captured) might provide insights.")

Imported necessary libraries for the agent setup.

API keys seem to be set. Proceeding with agent initialization.

LLM Initialized.
SQL Database loaded from: /content/drive/My Drive/Chinook.db

Available Tools:
- sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
- sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
- sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.
- sql_db_query_checker: Use this tool to double check if your quer